#Environment Setup

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Put Data in DataFrame (Articles)

In [12]:
import pandas as pd
import glob, os

# Place dataset path here
path = "../Datasets/CoAID/"


dfFalse = pd.read_csv(path+"NewsFakeCOVID-19.csv", usecols=['title'])
dfFalse['label']=0
dfFalseJuly = pd.read_csv(path+"NewsFakeCOVID-19-JULY.csv", usecols=['title'])
dfFalseJuly['label']=0
dfTrue = pd.read_csv(path+"NewsRealCOVID-19.csv", usecols=['title'], nrows=len(dfFalse.values))
dfTrue['label']=1
dfTrueJuly = pd.read_csv(path+"NewsRealCOVID-19-JULY.csv", usecols=['title'], nrows=len(dfFalseJuly.values))
dfTrueJuly['label']=1

dfTotal = pd.concat([dfTrue, dfFalse, dfTrueJuly, dfFalseJuly])

X = dfTotal['title'].values
y = dfTotal['label'].values
print("len(X)", len(X))
print("len(y)", len(y))

len(X) 1676
len(y) 1676


# Finetune BERT on CoAID

In [13]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

tokenizer("Attention is all you need")

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/resolve/main/voc

{'input_ids': [101, 1335, 5208, 2116, 1110, 1155, 1128, 1444, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [15]:
from sklearn.utils import shuffle
dfTotal = shuffle(dfTotal)
from datasets import Dataset

train_dataset = Dataset.from_pandas(dfTotal[:1257])
eval_dataset = Dataset.from_pandas(dfTotal[1257:])

In [16]:
def tokenize_data(example):
    return tokenizer(example['title'], padding='max_length')

train_dataset = train_dataset.map(tokenize_data, batched=True)
eval_dataset = eval_dataset.map(tokenize_data, batched=True)

from transformers import AutoModelForSequenceClassification
import torch
#model_path = '/content/drive/MyDrive/CoVerifi&MedVerifi/CoVerifi/CoVerifi/ML_Model/weights/finetune_bert.pt'


model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

# try:
#     model.load_state_dict(torch.load(model_path))
# except:
#     pass

from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer", num_train_epochs=3)

from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file https://huggingface.co/bert-base-cased/resolve/

In [17]:
#Train the model on 75% of dfTotal 
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, title. If __index_level_0__, title are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1257
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 474


Step,Training Loss


In [18]:
model_path = '../models/finetune_bert_on_CoAID.pt'
torch.save(model.state_dict(),model_path)

In [19]:
path_to_external_validation_dataset = ".../Datasets/External_Validation_Datasets//" # Replace with appropriate path
dfTotal_external = pd.read_csv(path_to_external_validation_dataset+"External_Validation1.csv", usecols=['title', 'label'])

def binaryLabel(label):
  if label == "TRUE":
    return 1
  return 0
dfTotal_external = dfTotal_external.dropna()
dfTotal_external['label'] = dfTotal_external['label'].apply(lambda label: binaryLabel(str(label)))
dataset_external = Dataset.from_pandas(dfTotal_external)

In [20]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
    
dataset_external = dataset_external.map(tokenize_data, batched=True)
trainer_eval = Trainer(
    model=model,
    args=training_args,
    eval_dataset=dataset_external,
    compute_metrics=compute_metrics,
)
#Evaluate without re-train the model
trainer_eval.evaluate()

  0%|          | 0/8 [00:00<?, ?ba/s]

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, title. If __index_level_0__, title are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 7057
  Batch size = 8


{'eval_accuracy': 0.9223466062066034,
 'eval_loss': 0.4804149270057678,
 'eval_runtime': 132.2253,
 'eval_samples_per_second': 53.371,
 'eval_steps_per_second': 6.678}